## Importing Libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Data preprocessing

In [5]:
batch_size = 32 #instead of processing one image at a time, the images are processed in a batch of 32
img_height = 180
img_width = 180
validation_split = 0.2

# "tf.keras.utils.image_dataset_from_directory" is a library function which loads images from a directory
# we have two directories here, test and train, each directory here has four classes
# label_mode defines how the lables are represented by the library function 
# seed ensures the random split is consistent and no images appear in both subsets if you run your code multiple times.

train_ds = tf.keras.utils.image_dataset_from_directory(
    r"D:\Study stuff\Fruit insights project\model\Model-1-fruit-freshness\train",
    validation_split=validation_split,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',  
    shuffle=True,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    r"D:\Study stuff\Fruit insights project\model\Model-1-fruit-freshness\train",
    validation_split=validation_split,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=True,
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    r"D:\Study stuff\Fruit insights project\model\Model-1-fruit-freshness\test",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',  
    shuffle=False,
)


Found 8733 files belonging to 4 classes.
Using 6987 files for training.
Found 8733 files belonging to 4 classes.
Using 1746 files for validation.
Found 2570 files belonging to 4 classes.


### Normalizing pixel values

In [6]:
# we are normalizing pixel values because if data is collected from multiple resources and all images have different lighting conditions, it helps to standardize the input 

# we are creating a normalization layer, it rescales the input images by multiplying each image by 1/255. The original pixel values range from 0 to 255, and by this multiplication we convert them to floating point values between 0.0 and 1.0
normalization_layer = tf.keras.layers.Rescaling(1./255)

# here the map() applies the lambda function to each batch
# x is a batch of images
# y is a batch of labels for each image in the batch
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))


### Optimizing data set loading

In [7]:
# tf.data.AUTOTUNE makes sure the data pipeline has the right number of threads working in parallel so training is smooth and efficient.

# data pipeline includes a number of processes such as, loading them from the disk, resizing them, shuffling them, normalizing them, batch them and feed them to the model.
AUTOTUNE = tf.data.AUTOTUNE

# prefetch() prepares the next batch of data while the model is being trained on the current one
# shuffle() is used because it randomizes the order of images so that the machine is not focusing on a specific order during training
# cache() places the data in cache memory because it speeds up the training process by avoiding the need to read data from disk repeatedly
# prefetch(buffer_size=AUTOTUNE) decides how many batches of prefetched data should be prepared ahead of time (in RAM) to keep the training pipeline smooth.

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


## CNN model
##### Convolutional Neural Network

In [9]:
# the "layers" module provides building blocks for processing data (like images)
# each layer in the pipeline has a specific role: - detect patterns, keep useful signals, shrink the data while preserving important information, flatten it into 1 dimension, and finally combine the features to make the final decision.

# The "models" module provides a container to organize and connect the layers together to build a neural network

from tensorflow.keras import layers, models


### Explaining the model

I am using a **Sequential model**, which means the output of one layer is directly passed as input to the next. Let me explain step by step:

##### 1. Input Layer
- The model takes an image of size **180 × 180** with **3 color channels (RGB)**.

---

##### 2. Convolution + Pooling Layers
- **Conv2D(32, 3×3, ReLU):** Detects very basic features like **edges, corners, and lines**.  
- **MaxPooling(2×2):** Shrinks the image while keeping the strongest signals.  

- **Conv2D(64, 3×3, ReLU):** Detects more complex features like **textures, curves, and shapes**.  
- **MaxPooling(2×2):** Shrinks again to focus on key features.  

- **Conv2D(128, 3×3, ReLU):** Detects high-level, detailed features like **eyes, leaves, or object parts**.  
- **MaxPooling(2×2):** Reduces the size but keeps the important signals.  

At this stage, the image has been turned into a set of **2D feature maps** (many small grids that represent different learned patterns).

---

##### 3. Flatten Layer
- The 2D feature maps are **flattened into a 1D vector**.  
- This means all the detected features are listed out in a row so we can make decisions.

---

##### 4. Dense(128, ReLU)
- This is a **fully connected layer**.  
- It learns **combinations of features**.  
- Example: *“If this edge + this texture + this color appear together → it might mean ‘fresh fruit’.”*  
- ReLU ensures we only keep the useful signals.

---

##### 5. Dense(4, Softmax)
- This is the **output layer** with 4 neurons (because we have 4 classes).  
- Each neuron acts like a **judge** that votes for one class.  
- **Softmax** converts these scores into **probabilities that add up to 1**.  


In [10]:
model = models.Sequential([
    layers.Input(shape=(180, 180, 3)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')  
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 51200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     6,553,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,647,492 (25.36 MB)

 Trainable params: 6,647,492 (25.36 MB)

 Non-trainable params: 0 (0.00 B)

#### compilation of the model

In [11]:
# here the neural network gets ready for training
# optimizer is like an engine and 'adam' (Adaptive moment estimation) is used so that the machine learns from it's mistake. It adjusts learning rate for each parameter.
# the loss function measures how far the model's predictions are correct.
# metrics tell us how well the model is performing, accuracy will tell us the percentage of correct predictions.
# Categorical crossentropy focuses on the correct class and checks how confident the model is about predicting it.
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


### Training the model

In [12]:
# epochs means that the model will go through the entire training dataset 10 times and each time it will try to improve
epochs = 10

# model.fit trains the neural network, it takes the training dataset and feeds it to the model in batches.
# after each epoch the model will test on validation data to check if it's learning from its mistakes or it's learning the patterns to predict unseen data.
# we are storing the training process in history
# at the starting epochs will be 1/10 and in the end it will be 10/10
history = model.fit(
    train_ds,
    validation_data=val_ds,  
    epochs=epochs
)


Epoch 1/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 608s 2s/step - accuracy: 0.7835 - loss: 0.5693 - val_accuracy: 0.8562 - val_loss: 0.3883
Epoch 2/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 334s 2s/step - accuracy: 0.8848 - loss: 0.3174 - val_accuracy: 0.9044 - val_loss: 0.2506
Epoch 3/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 319s 1s/step - accuracy: 0.9128 - loss: 0.2330 - val_accuracy: 0.9107 - val_loss: 0.2368
Epoch 4/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.9402 - loss: 0.1575 - val_accuracy: 0.9238 - val_loss: 0.1986
Epoch 5/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 305s 1s/step - accuracy: 0.9513 - loss: 0.1361 - val_accuracy: 0.9525 - val_loss: 0.1430
Epoch 6/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 329s 2s/step - accuracy: 0.9775 - loss: 0.0675 - val_accuracy: 0.9330 - val_loss: 0.2167
Epoch 7/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 288s 1s/step - accuracy: 0.9712 - loss: 0.0866 - val_accuracy: 0.9336 - val_loss: 0.1836
Epoch 8/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 308s 1s/step - accuracy: 0.9887 - loss: 0.0379 - val_accu